In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()
strategy.connect()
quarterly_sim = strategy.retrieve("quarterly_sim")
weekly_sim = strategy.retrieve("weekly_sim")
strategy.disconnect()

In [5]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [ ]:
trades = []
for number_of_training_weeks in tqdm(range(14,69,14)):
    for weekly_model_training_year in tqdm(range(1,6)):
        for weekly_category_training_year in tqdm(range(1,5)):
            current_weekly_sim = weekly_sim[(weekly_sim["mty"]==weekly_model_training_year) & \
                                           (weekly_sim["tw"]==number_of_training_weeks) & \
                                           (weekly_sim["cty"]==weekly_category_training_year)]
            for quarterly_category_training_year in tqdm(range(1,5)):
                for quarterly_model_training_year in tqdm(range(3,6)):
                    current_sim = quarterly_sim[(quarterly_sim["model_training_year"]==quarterly_model_training_year) & (quarterly_sim["category_training_year"]==quarterly_category_training_year)]
                    final_sim = prices.merge(current_sim,on=["year","quarter","ticker"],how="left").dropna().reset_index()
                    final_sim = final_sim[["year","quarter","week","date","ticker","adjClose","quarterly_price_regression_prediction","score"]]
                    final_sim.rename(columns={"score":"quarterly_score"},inplace=True)
                    final_sim["quarterly_delta"] = (final_sim["quarterly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
                    final_sim = final_sim.merge(weekly_sim,on=["year","quarter","week","ticker"],how="left")
                    final_sim["weekly_delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
                    final_sim.rename(columns={"score":"weekly_score"},inplace=True)
                    start = final_sim["date"].min()
                    end = final_sim["date"].max()
                    date = start
                    seats = 11
                    print(number_of_training_weeks,weekly_model_training_year
                          ,weekly_category_training_year
                          ,quarterly_model_training_year
                          ,quarterly_category_training_year
                         ,final_sim.index.size)
#                     for delta in tqdm(range(0,15,5)):
#                         d = delta/100
#                         for score in range(60,100,10):
#                             date = start
#                             while date <= end:
#                                 req = float(score/100)
#                                 if date.weekday() > 1:
#                                     date = date + timedelta(days=1)
#                                 try:
#                                     todays_sim = final_sim[(final_sim["date"]==date) & (final_sim["delta"] > 0) \
#                                                           & (final_sim["score"] >= req)].sort_values("delta",ascending=False)
#                                     if todays_sim.index.size < 1:
#                                         date = date + timedelta(days=1)
#                                     else:
#                                         for seat in range(min(seats,todays_sim.index.size)):
#                                             try:
#                                                 trade = todays_sim.iloc[seat]
#                                                 ticker = trade["ticker"]
#                                                 buy_price = trade["adjClose"]
#                                                 exit_price = buy_price * (1+delta)
#                                                 max_date = date + timedelta(days=30)
#                                                 exits = final_sim[(final_sim["ticker"]==ticker) & 
#                                                                     (final_sim["date"] > date) & \
#                                                                   (final_sim["date"] <= max_date) & \
#                                                                   (final_sim["adjClose"] >= exit_price)
#                                                                 ]
#                                                 if exits.index.size < 1:
#                                                     exits = final_sim[(final_sim["ticker"]==ticker) & 
#                                                                   (final_sim["year"] == date.year) & \
#                                                                   (final_sim["adjClose"] >= buy_price)].sort_values("date",ascending=False)
#                                                     if exits.index.size > 1:
#                                                         sell_trade = exits.iloc[0]
#                                                     else:
#                                                         exits = final_sim[(final_sim["ticker"]==ticker) & 
#                                                                   (final_sim["year"] == date.year)].sort_values("date",ascending=False)
#                                                         sell_trade = exits.iloc[exits.index.size -1]
#                                                 else:
#                                                     sell_trade = exits.iloc[0]
#                                                 trade["sell_date"] = sell_trade["date"]
#                                                 trade["sell_price"] = sell_trade["adjClose"]
#                                                 trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
#                                                 trade["score"] = score
#                                                 trade["d"] = d
#                                                 trade["qcty"] = quarterly_category_training_year
#                                                 trade["qmty"] = quarterly_model_training_year
#                                                 trade["wcty"] = weekly_category_training_year
#                                                 trade["wmty"] = weekly_model_training_year
#                                                 trade["tw"] = number_of_training_weeks
#                                                 trade["seat"] = seat
#                                                 trades.append(trade)
#                                                 date = sell_trade["date"] + timedelta(days=1)
#                                             except Exception as e:
#                                                 continue
#                                         date = date + timedelta(days=1)
#                                 except Exception as e:
#                                     date = date + timedelta(days=1)

  0%|                                                                                                                                             | 0/5 [00:00<?, ?it/s]

  0%|                                                                                                                                             | 0/4 [00:00<?, ?it/s]


  0%|                                                                                                                                             | 0/4 [00:00<?, ?it/s]



  0%|                                                                                                                                             | 0/3 [00:00<?, ?it/s]



 33%|████████████████████████████████████████████▎                                                                                        | 1/3 [00:04<00:09,  4.75s/it]

14 1 1 3 1 1913490






 67%|████████████████████████████████████████████████████████████████████████████████████████▋                                            | 2/3 [00:09<00:04,  4.89s/it]

14 1 1 4 1 1864987






100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.92s/it]



 25%|█████████████████████████████████▎                                                                                                   | 1/4 [00:14<00:44, 14.77s/it]

...

14 5 3 4 1 1864987






100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.02s/it]



 25%|█████████████████████████████████▎                                                                                                   | 1/4 [00:15<00:45, 15.08s/it]

14 5 3 5 1 2263407






  0%|                                                                                                                                             | 0/3 [00:00<?, ?it/s]



 33%|████████████████████████████████████████████▎                                                                                        | 1/3 [00:05<00:10,  5.28s/it]

14 5 3 3 2 2986242


In [ ]:
t = pd.DataFrame(trades)

In [ ]:
a = []
seats = 10
for category_training_year in tqdm(range(1,5)):
    for model_training_year in tqdm(range(3,6)):
        for delta in tqdm(range(0,15,5)):
            d = delta/100
            for score in tqdm(range(70,100,10)):
                score_trades = t[(t["score"]==score) & (t["d"]==d) & (t["cty"]==category_training_year) & (t["mty"]==model_training_year)]
                cash = []
                for seat in range(seats):
                    initial = float(100 / seats)
                    seat_trades = score_trades[score_trades["seat"]==seat]
                    for delta in seat_trades["delta"]:
                        initial = initial * (1+delta)
                        cash.append(initial)
                    analysis = {}
                    analysis["score"] = score
                    analysis["trades"] = seat_trades.index.size
                    if seat_trades.index.size > 0:
                        analysis["winrate"] = seat_trades[score_trades["delta"] > 0].index.size / seat_trades.index.size
                        seat_trades["hpr"] = (seat_trades["sell_date"] - seat_trades["date"])
                        seat_trades["hpr"] =[x.days for x in seat_trades["hpr"]]
                        analysis["hpr"] = seat_trades["hpr"].mean()
                        analysis["seat"] = seat
                        analysis["d"] = d
                        analysis["cty"] = category_training_year
                        analysis["mty"] = model_training_year
                        analysis["pv"] = initial
                        a.append(analysis)

In [ ]:
final = pd.DataFrame(a)

In [ ]:
final

In [ ]:
hl = final.pivot_table(index=["score","d","cty","mty"],columns="seat",values="pv")

In [ ]:
report = final.merge(hl.reset_index(),on=["score","d","cty","mty"]).drop(["pv","seat"],axis=1).groupby(["score","d"]).mean()
report["pv"] = [sum([row[1][x] for x in range(seats)]) for row in report.iterrows()]

In [ ]:
report.sort_values("pv",ascending=False).reset_index()

In [ ]:
score = report.sort_values("pv",ascending=False).reset_index().iloc[0]["score"].item()
d = report.sort_values("pv",ascending=False).reset_index().iloc[0]["d"].item()
cty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["cty"].item()
mty = report.sort_values("pv",ascending=False).reset_index().iloc[0]["mty"].item()
benchmark = t[(t["score"]==score) & (t["d"]==d) & (t["mty"]==mty) & (t["cty"]==cty)]

In [ ]:
mty

In [ ]:
timelines =[]
for seat in tqdm(range(seats)):
    seat_trades = benchmark[benchmark["seat"]==seat]
    initial = float(100/seats)
    timeline = []
    for row in seat_trades.iterrows():
        initial = initial * (1+row[1]["delta"])
        timeline.append(initial)
    seat_trades["pv"] = timeline
    timelines.append(seat_trades)

In [ ]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()
sp5.rename(columns={"Symbol":"ticker"},inplace=True)
benchmark = benchmark.merge(sp5,on="ticker",how="left")

In [ ]:
sp5.columns

In [ ]:
# industry check
benchmark.groupby(["year","quarter","GICS Sector"]).mean().sort_values("delta",ascending=False).reset_index()

In [ ]:
benchmark.groupby(["year","quarter","ticker"]).mean().sort_values("delta",ascending=False).reset_index().merge(sp5[["ticker","Security"]],on="ticker",how="left").head(20)